In [78]:
# -*- coding: utf-8 -*-
import os
import sys
import time
import pandas as pd
import numpy as np
import torch
from torch.optim import Adagrad
from sklearn.metrics import log_loss, roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

sys.path.append("/Users/wzq/Desktop/game")
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *
from deepctr_torch.callbacks import EarlyStopping, ModelCheckpoint

from utils import save_zip_file

In [49]:
data = pd.read_csv('./data/train_processed.csv')
test_data = pd.read_csv('./data/test_processed.csv')
print(data.shape)
print(test_data.shape)
data = pd.concat([data, test_data], axis=0)
print(data.shape)
data

(745889, 28)
(223553, 25)
(969442, 28)


,Unnamed: 0,user_id,product_id,product_name,rating,votes,helpful_votes,ID,user_rating_mean,user_rating_count,...,product_rating_mean,product_rating_count,product_rating_std,product_rating_var,product_votes_mean,product_votes_max,product_helpful_votes_mean,product_helpful_votes_max,product_votes_helpful_rate,product_helpful_rating
0,0,1813,154533,Beautiful Thing,5.0,10.0,8.0,0,4.251799,556,...,4.555556,9.0,0.527046,0.277778,4.444444,10.0,4.000000,8.0,0.900000,4.100000
1,1,1916,154533,Beautiful Thing,4.0,2.0,2.0,681987,4.458700,112483,...,4.555556,9.0,0.527046,0.277778,4.444444,10.0,4.000000,8.0,0.900000,4.100000
2,2,506,154533,Beautiful Thing,5.0,6.0,6.0,288065,4.785417,480,...,4.555556,9.0,0.527046,0.277778,4.444444,10.0,4.000000,8.0,0.900000,4.100000
3,3,923,154533,Beautiful Thing,5.0,9.0,8.0,365036,4.299320,147,...,4.555556,9.0,0.527046,0.277778,4.444444,10.0,4.000000,8.0,0.900000,4.100000
4,4,1560,154533,Beautiful Thing,4.0,6.0,5.0,237732,4.143437,739,...,4.555556,9.0,0.527046,0.277778,4.444444,10.0,4.000000,8.0,0.900000,4.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223548,223548,1916,214704,"Godzilla, King of the Monsters",NaN,NaN,NaN,223548,4.458700,112483,...,3.933333,15.0,1.222799,1.495238,5.266667,24.0,3.866667,21.0,0.734177,2.887764
223549,223549,1502,169489,Houses of the Holy,NaN,NaN,NaN,223549,4.310484,28404,...,4.451613,31.0,0.809885,0.655914,4.419355,46.0,2.225806,15.0,0.503650,2.242053
223550,223550,874,150908,Frankenstein and the Monster from Hell,NaN,NaN,NaN,223550,4.414830,4855,...,3.125000,8.0,1.246423,1.553571,4.750000,10.0,3.500000,8.0,0.736842,2.302632
223551,223551,922,198737,X-Men,NaN,NaN,NaN,223551,4.636000,1250,...,4.118280,93.0,0.942354,0.888032,8.010753,196.0,5.096774,141.0,0.636242,2.620221


In [50]:
sparse_features = ['user_id', 'product_id']

dense_features = ['user_rating_mean', 'user_votes_mean', 'user_helpful_votes_mean'] \
                + ['product_rating_mean', 'product_votes_mean','product_helpful_votes_mean'] \
                + ['user_votes_helpful_rate', 'user_helpful_rating'] \
                + ['product_votes_helpful_rate', 'product_helpful_rating']
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['rating']
data

,Unnamed: 0,user_id,product_id,product_name,rating,votes,helpful_votes,ID,user_rating_mean,user_rating_count,...,product_rating_mean,product_rating_count,product_rating_std,product_rating_var,product_votes_mean,product_votes_max,product_helpful_votes_mean,product_helpful_votes_max,product_votes_helpful_rate,product_helpful_rating
0,0,1813,154533,Beautiful Thing,5.0,10.0,8.0,0,4.251799,556,...,4.555556,9.0,0.527046,0.277778,4.444444,10.0,4.000000,8.0,0.900000,4.100000
1,1,1916,154533,Beautiful Thing,4.0,2.0,2.0,681987,4.458700,112483,...,4.555556,9.0,0.527046,0.277778,4.444444,10.0,4.000000,8.0,0.900000,4.100000
2,2,506,154533,Beautiful Thing,5.0,6.0,6.0,288065,4.785417,480,...,4.555556,9.0,0.527046,0.277778,4.444444,10.0,4.000000,8.0,0.900000,4.100000
3,3,923,154533,Beautiful Thing,5.0,9.0,8.0,365036,4.299320,147,...,4.555556,9.0,0.527046,0.277778,4.444444,10.0,4.000000,8.0,0.900000,4.100000
4,4,1560,154533,Beautiful Thing,4.0,6.0,5.0,237732,4.143437,739,...,4.555556,9.0,0.527046,0.277778,4.444444,10.0,4.000000,8.0,0.900000,4.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223548,223548,1916,214704,"Godzilla, King of the Monsters",NaN,NaN,NaN,223548,4.458700,112483,...,3.933333,15.0,1.222799,1.495238,5.266667,24.0,3.866667,21.0,0.734177,2.887764
223549,223549,1502,169489,Houses of the Holy,NaN,NaN,NaN,223549,4.310484,28404,...,4.451613,31.0,0.809885,0.655914,4.419355,46.0,2.225806,15.0,0.503650,2.242053
223550,223550,874,150908,Frankenstein and the Monster from Hell,NaN,NaN,NaN,223550,4.414830,4855,...,3.125000,8.0,1.246423,1.553571,4.750000,10.0,3.500000,8.0,0.736842,2.302632
223551,223551,922,198737,X-Men,NaN,NaN,NaN,223551,4.636000,1250,...,4.118280,93.0,0.942354,0.888032,8.010753,196.0,5.096774,141.0,0.636242,2.620221


In [51]:
# 对稀疏的类别特征进行简单的处理，处理为连续整数类别
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [52]:
# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].max() + 1, embedding_dim=4) for feat in sparse_features] \
                       + [DenseFeat(feat, 1, )for feat in dense_features]
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
feature_names

['user_id',
 'product_id',
 'user_rating_mean',
 'user_votes_mean',
 'user_helpful_votes_mean',
 'product_rating_mean',
 'product_votes_mean',
 'product_helpful_votes_mean',
 'user_votes_helpful_rate',
 'user_helpful_rating',
 'product_votes_helpful_rate',
 'product_helpful_rating']

In [53]:
# 3.generate input data for model
test_predict = data.iloc[-test_data.shape[0]:, :] # (223553, 28)
train, test = train_test_split(data.iloc[:-test_data.shape[0], :], test_size=0.2, random_state=2020)  # 删除测试数据集 (745889, 28)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}
test_predict_input = {name: test_predict[name] for name in feature_names}

In [79]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = False
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = DCN(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns, cross_num=2,
            dnn_hidden_units=(128, 128), l2_reg_linear=0.00001, l2_reg_embedding=0.00001, l2_reg_cross=0.00001, l2_reg_dnn=0, 
            dnn_dropout=0.5, dnn_activation='relu', dnn_use_bn=False, task='regression')
model.compile(Adagrad(model.parameters(),0.001), loss="mse", metrics=["mse"])

es = EarlyStopping(monitor='val_mse', min_delta=0, verbose=1, patience=5, mode='min')
mdckpt = ModelCheckpoint(filepath='best_models/DCN_model.ckpt', monitor='val_mse', verbose=1, save_best_only=True, mode='min')
history = model.fit(train_model_input, train[target].values, batch_size=128, epochs=100, verbose=2, validation_split=0.2, callbacks=[es,mdckpt])
model = torch.load('best_models/DCN_model.ckpt')

pred_ans = model.predict(test_model_input, 128)
print("")
print("test mse", round(mean_squared_error(test[target].values, pred_ans), 4))
print("test rmse", round(np.sqrt(mean_squared_error(test[target].values, pred_ans)), 4))

print(f"=========infer========={time.strftime('%Y%m%d%H%M', time.localtime())}")
outputs = model.predict(test_predict_input, 128)
test_data['rating'] =  outputs
submission = test_data[['ID', 'rating']]
save_zip_file(submission, )
print(f"=========infer done========={time.strftime('%Y%m%d%H%M', time.localtime())}")
submission

cpu
Train on 477368 samples, validate on 119343 samples, 3730 steps per epoch
Epoch 1/100
27s - loss:  0.7356 - mse:  0.7356 - val_mse:  0.6394
Epoch 00001: val_mse improved from inf to 0.63936, saving model to best_models/DCN_model.ckpt
Epoch 2/100
26s - loss:  0.6236 - mse:  0.6235 - val_mse:  0.6398
Epoch 00002: val_mse did not improve from 0.63936
Epoch 3/100
24s - loss:  0.5977 - mse:  0.5977 - val_mse:  0.6531
Epoch 00003: val_mse did not improve from 0.63936
Epoch 4/100
25s - loss:  0.5611 - mse:  0.5611 - val_mse:  0.6898
Epoch 00004: val_mse did not improve from 0.63936
Epoch 5/100
27s - loss:  0.5283 - mse:  0.5283 - val_mse:  0.7227
Epoch 00005: val_mse did not improve from 0.63936
Epoch 6/100
27s - loss:  0.5042 - mse:  0.5042 - val_mse:  0.7398
Epoch 00006: val_mse did not improve from 0.63936
Epoch 00006: early stopping

test mse 0.6393
test rmse 0.7995
=========infer=========202410111806
=========infer done=========202410111806


,ID,rating
0,0,4.996349
1,1,1.689050
2,2,4.434983
3,3,3.742712
4,4,4.088385
...,...,...
223548,223548,3.940505
223549,223549,4.447436
223550,223550,3.331333
223551,223551,4.582328


In [73]:
print(f"=========infer========={time.strftime('%Y%m%d%H%M', time.localtime())}")
outputs = model.predict(test_predict_input, 128)
test_data['rating'] =  outputs
submission = test_data[['ID', 'rating']]
save_zip_file(submission, )
print(f"=========infer done========={time.strftime('%Y%m%d%H%M', time.localtime())}")
submission

=========infer=========202410111727
=========infer done=========202410111727


,ID,rating
0,0,4.996349
1,1,1.689050
2,2,4.434983
3,3,3.742712
4,4,4.088385
...,...,...
223548,223548,3.940505
223549,223549,4.447436
223550,223550,3.331333
223551,223551,4.582328
